## Check Test Dataset prediction results

In [1]:
import numpy as np

Y_pred_dir = "/Users/achs/PhD/code/CT-MRI_LandmarkDetection/Spartan/trained_models/divided_dataset/straight_model/mean_two_landmarks/"

Y_test_pred_file = "bestVal_straight_model_divided_176_88_48_mean_two_landmarks_MTL_p.npy"
# test_y_pred_file = "bestVal_first_model_y_test.npy"

Y_test_pred_path = Y_pred_dir + Y_test_pred_file
Y_test_pred = np.load(Y_test_pred_path)


In [2]:
import tensorflow as tf

import Spartan.support_modules as supporter
import Functions.MyDataset as MyDataset

# Get the Test Dataset Prediction Results
datasets_dir = "/Volumes/Shawn_HDD/PhD/Project/Date/augmentation_from_matlab/Rescaled/"
size = (176, 176, 48)
with_res = True

str_size = str(size[0]) + "_" + str(size[1]) + "_" + str(size[2])
if with_res:
    str_size = str_size + "_PD"

dataset_path = datasets_dir + str_size + "/"
pat_splits = MyDataset.get_pat_splits(static=True)
X_test, Y_test, res_test = \
    supporter.load_dataset(dataset_path, size, pat_splits, with_res=with_res, only_test=True)

# Because the resolution is for 4 landmark points, so we repeat it to make the calculation easier.
res_test_R4 = np.repeat(res_test, 4, axis=1)

X_test Shape:  (200, 176, 176, 48)
Y_test Shape:  (200, 4, 3)
X_test_reshape Shape:  (200, 176, 176, 48, 1)
Y_test Shape:  (200, 4, 3)


2023-04-27 15:21:15.013981: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
res_test_R4 = np.repeat(res_test, 4, axis=1)

In [10]:
Y_test_mapped = Y_test * res_test_R4 / 0.15
Y_MTL_test_mapped = np.mean(Y_test_mapped.reshape((400, 2, 3)), axis=1).reshape((400, 1, 3))
# Y_test_pred_mapped = Y_test_pred * res_test_R4 / 0.15
Y_MTL_test_mapped_pred = Y_test_pred

print("Y_test_mapped Shape: ", Y_test_mapped.shape)
print("Y_MTL_test_mapped Shape: ", Y_MTL_test_mapped.shape)

Y_test_mapped Shape:  (200, 4, 3)
Y_MTL_test_mapped Shape:  (400, 1, 3)


## Check the difference between Predictions and Ground Truth

### Ground Truth points and Prediction points distance

In [11]:
# 1. Total error distance
# 2. Maximum one point's error distance
# 3. If it can involve the Ground Truth points
# 4. If it can involve the Target Area

# 1. Total error distance
err_diff = Y_MTL_test_mapped - Y_MTL_test_mapped_pred
square_err_diff = tf.pow(err_diff, 2)
sum_square_err_diff = tf.reduce_sum(square_err_diff, axis=[1, 2])

In [12]:
min_err_idx = np.argmin(sum_square_err_diff, axis=0)
max_err_idx = np.argmax(sum_square_err_diff, axis=0)
print(f"Min[{min_err_idx}]: {sum_square_err_diff[min_err_idx]}")
print(f"Max[{max_err_idx}]: {sum_square_err_diff[max_err_idx]}")
print(f"Mean: {np.mean(sum_square_err_diff)}")

Min[139]: 200.25697326660156
Max[52]: 444347.5
Mean: 156316.28125


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

mu = np.mean(sum_square_err_diff)
sigma = np.std(sum_square_err_diff)

# Create the bins and histogram
count, bins, ignored = plt.hist(sum_square_err_diff, 20)

plt.show()

In [ ]:
idx = max_err_idx

pat_names = MyDataset.get_pat_names()
test_pat_idx = MyDataset.get_pat_splits(static=True)[2] # 0: train, 1: val, 2: test

test_pt_idx = np.floor(idx / 50).astype(int)
pt_aug_id = idx % 50 + 1

pat_name = pat_names[test_pat_idx[test_pt_idx]]

print(f"Patient {pat_name}, Aug Id: {pt_aug_id}")

In [ ]:
## Check the cropped results
import numpy as np
import Functions.MyDataset as MyDataset
import Functions.Visualization as Visualization

aug_dir = "/Volumes/Shawn_HDD/PhD/Project/Date/augmentation_from_matlab/original_augmentation_data/"
pt_aug_path = aug_dir + pat_name + "_aug_" + str(pt_aug_id) + ".mat"

pixel_space = [0.15, 0.15, 0.15]

pt_aug_volume, pt_aug_pts, _ = MyDataset.load_mat_data(pt_aug_path)

# Ground Truth
Visualization.show_pts(pt_aug_volume, pt_aug_pts, pixel_space)
# Prediction on original volumes
Visualization.show_pts(pt_aug_volume, Y_test_pred_mapped[idx], pixel_space)

In [ ]:
# Ground Truth
Visualization.show_pts(X_test[idx,:,:,:,0], Y_test[idx], pixel_space)
# Prediction
Visualization.show_pts(X_test[idx,:,:,:,0], Y_test_pred[idx], pixel_space)

### Ground Truth points to the Cropped Volume's border distance

In [ ]:
import Functions.MyCrop as MyCrop

# anchor = np.average(pt_aug_pts[0:2], axis=0).astype(int)
anchor = np.average(Y_test_pred_mapped[idx][0:2], axis=0).astype(int)

d = MyCrop.distance_from_border(pt_aug_volume.shape, pt_aug_pts[0:2], anchor)

# attention: this is (row, column, slice) same as (y, x, z)
print(d)